In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import glob
from lxml import objectify
from lxml import etree
import xml.dom.minidom

import sys
sys.path.append(r'C:\Users\User\Documents\Code\blimp')
sys.path.append(r'C:\Users\User\Documents\Code\blimp\sdk')

from parse_markpoints import parse_markpoints
from prairie_interface import prairie_interface
from SLM_sdk import SLM_sdk

In [5]:
class b_52(parse_markpoints, prairie_interface):
    
    def __init__(self, naparm_path, naparm_hz=1000):
        
            
        ''' class to automatically run the naparam experiment in naparm_path '''
    
        
        self.naparm_path = naparm_path
        self.naparm_hz = naparm_hz
        self.get_paths()
        self.get_masks()
        self.trigger_times(self.trigger_path)
        
        self.n_trials = self.num_stims / self.num_masks
        
        #check that the number of trials logic is correct
        assert self.n_trials.is_integer() == True and self.n_trials > 0
        self.n_trials = int(self.n_trials)
        
        #init the SLM sdk class
        self.sdk = SLM_sdk()
        self.sdk.SLM_connect()
        
        #inherits the attributes of classes parse_markpoints and prairie_interface
        super(b_52, self).__init__(self.xml_path, self.gpl_path)
        
        #check that the same number of gpl items and xml items have been found
        assert len(self.laser_powers) == len(self.spiral_sizes) 
        
        #from the parse_markpoints class
        self.markpoints_strings = self.build_strings()
        
        if self.dummy:
            self.dummy_string = self.build_dummy()
          
    def get_paths(self):
        
        for file in os.listdir(naparm_path):
            
            file_path = os.path.join(naparm_path, file)

            if file == 'ToSLM_AllTrials.dat':
                self.trigger_path = file_path
            elif file.endswith('.gpl'):
                self.gpl_path = file_path
            elif file.endswith('.xml'):
                self.xml_path = file_path
            elif file == 'PhaseMasks':
                self.mask_path = file_path
                
    def get_masks(self):
        
        self.mask_files = []

        for file in os.listdir(self.mask_path):
            file_path = os.path.join(self.mask_path, file)
            
            if file.endswith('.tif') or file.endswith('.tiff'):
                self.mask_files.append(file_path)
        
        # i think this puts the files in the order expected by naparm, but this is a potential error source
        self.mask_files.sort()
        
        self.num_masks = len(self.mask_files)
                
        
    def trigger_times(self, dat_path):
    
        ''' 
        function to detect edges signalling trigger onset in dat files
        input: dat file path
        output: times of trigger onsets in dat file (seconds)

        '''

        arr = np.fromfile(dat_path, dtype=float)

        #edge detect the SLM trigger dat file      
        edges = [i for i,j in enumerate(arr) if arr[i-1] == 0 and arr[i] == 5]

        # above list doesnt hold for triggers sent at sample 0
        if arr[0] == 5:
            edges[0] = 0

        self.triggers = np.array(edges) / self.naparm_hz
        self.num_stims = len(self.triggers)

        self.sleep_times = np.diff(self.triggers)
        
    
    def drop_naparm(self):
        
        ''' runs the experiment outlined in the naparm folder
            currently only works when the first shot is at time 0     
            and each patterns has the same number of reps
        '''
        
        #repeat the masks paths n_trials times. So can be iterated through on each trial
        rep_masks = np.tile(self.mask_files, self.n_trials)
        
        rep_strings = np.tile(self.markpoints_strings, self.n_trials)
                
        #open the uncaging shutter and set the laser power to 0
        self.pl.SendScriptCommands('-SetLaserPower Uncaging 0')
        time.sleep(0.5)
        self.pl.SendScriptCommands('-OverrideHardShutter Uncaging open')
        

        #load the first phase mask and drive galvos to position      
        self.sdk.load_mask(self.mask_files[0])
        self.pl.SendScriptCommands(self.dummy_string)
                
        time.sleep(2)
        
        
        
        for trial, (mask,string,sleep_time) in enumerate(zip(rep_masks,rep_strings,self.sleep_times)):
            
            
            t1 = time.time()
            
            self.sdk.load_mask(self.mask_files[0])
            
            t2 = time.time()
            
            #the time taken to load the mask
            t_mask = t2-t1
            
            if trial != 0:
                #sleep minus the time taken to load the phase mask
                time.sleep(sleep_time-t_mask-t_markpoints)
            
            

            t3 = time.time()
            
            #currently assumes that all masks have the same number of reps
            #for _ in range(int(self.repetitions[0])):

            #fire markpoints
            self.pl.SendScriptCommands(rep_strings[trial])

                #assumes all durations are the same
                #time.sleep(0.1)
            
            t4 = time.time()
            
            t_markpoints = t4-t3
            
            print('t_markpoints is {}'.format(t_markpoints))
            
             
            
        self.pl.SendScriptCommands('-OverrideHardShutter Uncaging close')
        

In [6]:
naparm_path = r'F:\Data\jrowland\2019-01-16\naparm\2019-01-16_plasticSlide_NAPARM_001'
anp = b_52(naparm_path)


SLM connection established
PV connection successful


In [7]:
anp.drop_naparm()


t_markpoints is 0.0010030269622802734
t_markpoints is 0.0010025501251220703
t_markpoints is 0.001003265380859375
t_markpoints is 0.003007650375366211
t_markpoints is 0.0010025501251220703
t_markpoints is 0.001003265380859375
t_markpoints is 0.0010027885437011719
t_markpoints is 0.0010027885437011719
t_markpoints is 0.0010027885437011719
t_markpoints is 0.0010027885437011719
t_markpoints is 0.0010030269622802734
t_markpoints is 0.0010030269622802734
t_markpoints is 0.0010030269622802734
t_markpoints is 0.0010030269622802734
t_markpoints is 0.0010025501251220703
t_markpoints is 0.0010030269622802734
t_markpoints is 0.0010027885437011719
t_markpoints is 0.0010037422180175781
t_markpoints is 0.001003265380859375
t_markpoints is 0.0010030269622802734
t_markpoints is 0.003007650375366211
t_markpoints is 0.0010027885437011719
t_markpoints is 0.0010027885437011719
t_markpoints is 0.0010025501251220703
t_markpoints is 0.001003265380859375
t_markpoints is 0.0010030269622802734
t_markpoints is 0.

In [7]:
int(anp.durations[0]) / 1000

0.002